### Import the necessary packages and load the data

In [9]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from hmmlearn import hmm
import sklearn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [8]:
!pip install hmmlearn

In [10]:
data = pd.read_csv('ner_dataset.csv', encoding = "latin1")
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


### Data Preprocessing and Data Cleaning

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  47959 non-null    object
 1   Word        1048575 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [12]:
data.nunique()

Sentence #    47959
Word          35178
POS              42
Tag              17
dtype: int64

In [13]:
data.rename(columns={'Sentence #':'Sentence'}, inplace=True)

In [14]:
data = data.fillna(method = 'ffill')

In [15]:
data.isnull().sum()

Sentence    0
Word        0
POS         0
Tag         0
dtype: int64

In [16]:
print(data.head(10))
print(data.tail(10))

      Sentence           Word  POS    Tag
0  Sentence: 1      Thousands  NNS      O
1  Sentence: 1             of   IN      O
2  Sentence: 1  demonstrators  NNS      O
3  Sentence: 1           have  VBP      O
4  Sentence: 1        marched  VBN      O
5  Sentence: 1        through   IN      O
6  Sentence: 1         London  NNP  B-geo
7  Sentence: 1             to   TO      O
8  Sentence: 1        protest   VB      O
9  Sentence: 1            the   DT      O
                Sentence       Word  POS    Tag
1048565  Sentence: 47958     impact   NN      O
1048566  Sentence: 47958          .    .      O
1048567  Sentence: 47959     Indian   JJ  B-gpe
1048568  Sentence: 47959     forces  NNS      O
1048569  Sentence: 47959       said  VBD      O
1048570  Sentence: 47959       they  PRP      O
1048571  Sentence: 47959  responded  VBD      O
1048572  Sentence: 47959         to   TO      O
1048573  Sentence: 47959        the   DT      O
1048574  Sentence: 47959     attack   NN      O


In [17]:
' '.join(data[data['Sentence'] == 'Sentence: 1']['Word'].values)

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [18]:
' '.join(data[data['Sentence'] == 'Sentence: 2']['Word'].values)

'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "'

In [19]:
' '.join(data[data['Sentence'] == 'Sentence: 47959']['Word'].values)

'Indian forces said they responded to the attack'

In [20]:
data['Tag'].value_counts()

O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: Tag, dtype: int64

In [21]:
tags = list(set(data['Tag'].values))
words = list(set(data['Word'].values))
print('Number of unique words in the dataset:', len(words))
print('Number of unique tags in the dataset:', len(tags))

Number of unique words in the dataset: 35178
Number of unique tags in the dataset: 17


### Data Preparation - Split the data into train and test set

We use **GroupShuffleSplit** to spilt the data into train and test set as using **train_test_split** will make some parts of the sentence go to train set and others to test set

In [22]:
data.columns

Index(['Sentence', 'Word', 'POS', 'Tag'], dtype='object')

In [23]:
y = data['Tag']
x = data.drop(columns='Tag')

In [24]:
split_gs = GroupShuffleSplit(n_splits=3, test_size=0.3, random_state=42)
# n_splitsint, default=5 : Number of re-shuffling & splitting iterations.

In [25]:
x_train, x_test = next(split_gs.split(x, y, groups=x['Sentence']))

In [26]:
x.shape

(1048575, 3)

In [27]:
x_train.shape, x_test.shape

((734523,), (314052,))

In [28]:
train_data = data.iloc[x_train]
test_data = data.iloc[x_test]

In [29]:
train_data.shape, test_data.shape

((734523, 4), (314052, 4))

In [30]:
train_data.head()

,Sentence,Word,POS,Tag
24,Sentence: 2,Families,NNS,O
25,Sentence: 2,of,IN,O
26,Sentence: 2,soldiers,NNS,O
27,Sentence: 2,killed,VBN,O
28,Sentence: 2,in,IN,O


In [33]:
data_update = train_data.sample(frac=0.1, replace=False, random_state=42).reset_index(drop=True)

In [34]:
data_update.shape

(73452, 4)

In [35]:
data_update.head()

,Sentence,Word,POS,Tag
0,Sentence: 23582,the,DT,O
1,Sentence: 12377,of,IN,O
2,Sentence: 26652,.,.,O
3,Sentence: 9035,the,DT,O
4,Sentence: 16245,killed,VBN,O


In [36]:
data_update.Word = "UNKNOWN"

In [37]:
train_data['Word'].unique()

array(['Families', 'of', 'soldiers', ..., 'Soe', '3700', 'Bermel'],
      dtype=object)

In [38]:
train_data.update(data_update)

C:\Temp\ipykernel_12204\1501397837.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.update(data_update)


In [39]:
train_data['Word'].unique()

array(['UNKNOWN', 'the', 'illness', ..., 'Soe', '3700', 'Bermel'],
      dtype=object)

In [40]:
display(train_data.head().reset_index(drop=True))
display(train_data.tail().reset_index(drop=True))

,Sentence,Word,POS,Tag
0,Sentence: 31683,UNKNOWN,NNP,O
1,Sentence: 8392,UNKNOWN,DT,O
2,Sentence: 28070,UNKNOWN,NNS,O
3,Sentence: 40293,UNKNOWN,DT,O
4,Sentence: 44552,UNKNOWN,IN,O


,Sentence,Word,POS,Tag
0,Sentence: 47959,they,PRP,O
1,Sentence: 47959,responded,VBD,O
2,Sentence: 47959,to,TO,O
3,Sentence: 47959,the,DT,O
4,Sentence: 47959,attack,NN,O


In [42]:
print(words[:10])

['censored', 'Dragmulla', 'foster', 'analysts', 'tilt', 'Dick', 'Sirnak', 'Tanith', 'Dhabi', 'dominance']


In [43]:
word2id = {w:i for i,w in enumerate(words)}
tag2id = {t:i for i,t in enumerate(tags)}
id2tag = {i:t for i,t in enumerate(tags)}

In [44]:
print(word2id)

{'censored': 0, 'Dragmulla': 1, 'foster': 2, 'analysts': 3, 'tilt': 4, 'Dick': 5, 'Sirnak': 6, 'Tanith': 7, 'Dhabi': 8, 'dominance': 9, 'recommended': 10, 'Raffaele': 11, 'Majlis-e-Amal': 12, 'Warlords': 13, 'repressing': 14, 'budgets': 15, 'Krzyzewski': 16, 'clash': 17, 'discovering': 18, 'HELP': 19, 'event': 20, 'reversed': 21, 'stole': 22, 'supervise': 23, 'loses': 24, 'pork': 25, 'simplification': 26, 'Friedan': 27, 'methamphetamine': 28, 'crushed': 29, 'Marlins': 30, 'Asked': 31, 'grata': 32, 'Melli': 33, '-': 34, 'Accordance': 35, 'al-Talhi': 36, 'socialist': 37, 'Rahman': 38, 'Sibal': 39, 'catching': 40, 'avoid': 41, 'steam': 42, 'de-orbiting': 43, 'subcommittee': 44, 'sponsorship': 45, 'Port': 46, 'protect': 47, 'salivated': 48, 'Brett': 49, 'reshuffled': 50, 'Barbara': 51, 'Halliburton': 52, '46-member': 53, 'Barbour': 54, 'ploy': 55, 'Minster': 56, 'kidnappings': 57, 'Jalani': 58, 'betting': 59, 'gathering': 60, 'bailout': 61, 'countering': 62, 'spur': 63, 'reflects': 64, '1.

In [45]:
print(tag2id)

{'B-geo': 0, 'O': 1, 'I-geo': 2, 'I-tim': 3, 'I-nat': 4, 'I-per': 5, 'B-gpe': 6, 'I-gpe': 7, 'B-org': 8, 'I-eve': 9, 'B-eve': 10, 'I-art': 11, 'B-tim': 12, 'B-nat': 13, 'B-per': 14, 'I-org': 15, 'B-art': 16}


In [46]:
print(id2tag)

{0: 'B-geo', 1: 'O', 2: 'I-geo', 3: 'I-tim', 4: 'I-nat', 5: 'I-per', 6: 'B-gpe', 7: 'I-gpe', 8: 'B-org', 9: 'I-eve', 10: 'B-eve', 11: 'I-art', 12: 'B-tim', 13: 'B-nat', 14: 'B-per', 15: 'I-org', 16: 'B-art'}


### Hidden Markov Models

In [47]:
count_tags = dict(train_data['Tag'].value_counts())
print(count_tags)

{'O': 621981, 'B-geo': 26568, 'B-tim': 14380, 'B-org': 14062, 'I-per': 11998, 'B-per': 11835, 'I-org': 11799, 'B-gpe': 10912, 'I-geo': 5246, 'I-tim': 4692, 'B-art': 255, 'B-eve': 192, 'I-art': 174, 'I-eve': 149, 'B-nat': 130, 'I-gpe': 123, 'I-nat': 27}


In [48]:
train_data.head()

,Sentence,Word,POS,Tag
24,Sentence: 31683,UNKNOWN,NNP,O
25,Sentence: 8392,UNKNOWN,DT,O
26,Sentence: 28070,UNKNOWN,NNS,O
27,Sentence: 40293,UNKNOWN,DT,O
28,Sentence: 44552,UNKNOWN,IN,O


In [49]:
grouped_data = train_data.groupby('Tag')
for group_name, group_data in grouped_data:
    print(f"Group: {group_name}")
    display(group_data)

Group: B-art


,Sentence,Word,POS,Tag
13288,Sentence: 10857,UNKNOWN,IN,B-art
14210,Sentence: 21780,UNKNOWN,NNP,B-art
15551,Sentence: 46145,UNKNOWN,NNP,B-art
20690,Sentence: 8926,UNKNOWN,NN,B-art
25429,Sentence: 26264,UNKNOWN,NNP,B-art
...,...,...,...,...
1027190,Sentence: 46974,Arc,NNP,B-art
1033280,Sentence: 47247,Google,NNP,B-art
1041052,Sentence: 47608,Zim,NNP,B-art
1041448,Sentence: 47627,al-Jazeera,NNP,B-art


Group: B-eve


,Sentence,Word,POS,Tag
10337,Sentence: 26807,UNKNOWN,NNP,B-eve
21751,Sentence: 9008,UNKNOWN,NNP,B-eve
26762,Sentence: 1962,UNKNOWN,NNP,B-eve
35414,Sentence: 32870,UNKNOWN,NNP,B-eve
36708,Sentence: 8482,UNKNOWN,NNS,B-eve
...,...,...,...,...
1027168,Sentence: 46973,Armistice,NNP,B-eve
1027179,Sentence: 46974,World,NNP,B-eve
1041907,Sentence: 47649,II,NNP,B-eve
1041940,Sentence: 47651,World,NNP,B-eve


Group: B-geo


...

,Sentence,Word,POS,Tag
106,Sentence: 2393,UNKNOWN,NNP,I-per
116,Sentence: 39633,UNKNOWN,NNP,I-per
124,Sentence: 15431,UNKNOWN,NNP,I-per
246,Sentence: 22829,UNKNOWN,NNP,I-per
343,Sentence: 30836,UNKNOWN,NNP,I-per
...,...,...,...,...
1048135,Sentence: 47939,Karzai,NNP,I-per
1048157,Sentence: 47940,Richards,NNP,I-per
1048230,Sentence: 47943,Khin,NNP,I-per
1048231,Sentence: 47943,Nyunt,NNP,I-per


Group: I-tim


,Sentence,Word,POS,Tag
75,Sentence: 3864,UNKNOWN,CD,I-tim
522,Sentence: 27757,UNKNOWN,CD,I-tim
607,Sentence: 7878,UNKNOWN,NN,I-tim
769,Sentence: 27182,UNKNOWN,CC,I-tim
944,Sentence: 39400,UNKNOWN,CD,I-tim
...,...,...,...,...
1046766,Sentence: 47869,1992,CD,I-tim
1046767,Sentence: 47869,and,CC,I-tim
1046768,Sentence: 47869,2000,CD,I-tim
1047132,Sentence: 47886,18,CD,I-tim


Group: O


,Sentence,Word,POS,Tag
24,Sentence: 31683,UNKNOWN,NNP,O
25,Sentence: 8392,UNKNOWN,DT,O
26,Sentence: 28070,UNKNOWN,NNS,O
27,Sentence: 40293,UNKNOWN,DT,O
28,Sentence: 44552,UNKNOWN,IN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [54]:
count_tags_to_words = train_data.groupby('Tag').apply(lambda x: x.groupby('Word')['Tag'].count().to_dict()).to_dict()
display(count_tags_to_words)

{'B-art': {'300': 2,
  '@dalailama': 1,
  'A380s': 1,
  'AK-47': 1,
  'AM': 1,
  'Abu': 2,
  'Ai': 2,
  'Akash': 1,
  'Al': 1,
  'Anglo-French': 1,
  'Ansari': 1,
  'Arabic': 2,
  'Arc': 1,
  'Ariana': 1,
  'Atlantis': 1,
  'Axum': 1,
  'Azeri': 1,
  'BPA': 1,
  'Balad': 1,
  'Basilica': 1,
  'Beijing': 2,
  'Best': 1,
  'Bisphenol': 1,
  'Blades': 1,
  'Blueberry': 1,
  'Boeing': 1,
  'British': 1,
  'Buddhism': 1,
  'Cajun': 1,
  'Camp': 3,
  'Canal': 4,
  'Capitol': 1,
  'China': 1,
  'Chinook': 1,
  'Chung': 1,
  'Creole': 1,
  'Culture': 1,
  'Cy': 1,
  'Da': 1,
  'Date': 1,
  'Dejarte': 1,
  'Democrat': 1,
  'Destiny': 1,
  'Dextre': 2,
  'Didadgah': 1,
  'Dignity': 1,
  'Doha': 1,
  'Economics': 1,
  'El~Commercio': 1,
  'Emergency': 1,
  'Emmy': 1,
  'Endeavor': 1,
  'Endeavour': 1,
  'English': 10,
  'Facebook': 6,
  'Ferrari': 1,
  'For': 1,
  'Forbes': 3,
  'Fort': 1,
  'Four': 1,
  'Frankenstadion': 1,
  'French': 2,
  'GDP': 4,
  'Georgia': 1,
  'German': 1,
  'Google': 2,

In [55]:
count_init_tags = dict(train_data.groupby('Sentence').first().Tag.value_counts())
display(count_init_tags)

{'O': 26999,
 'B-geo': 1434,
 'B-per': 1059,
 'B-org': 982,
 'B-gpe': 803,
 'B-tim': 553,
 'I-per': 410,
 'I-org': 391,
 'I-geo': 188,
 'I-tim': 167,
 'B-art': 9,
 'B-eve': 8,
 'I-art': 8,
 'B-nat': 6,
 'I-eve': 5,
 'I-gpe': 4,
 'I-nat': 1}

In [56]:
count_tags_to_next_tags = np.zeros((len(tags), len(tags)),dtype=int)
print(count_tags_to_next_tags.shape)

(17, 17)


In [57]:
train_data['Sentence'][:5]

24    Sentence: 31683
25     Sentence: 8392
26    Sentence: 28070
27    Sentence: 40293
28    Sentence: 44552
Name: Sentence, dtype: object

In [58]:
sentences = list(train_data.Sentence)
print(sentences[:5])

['Sentence: 31683', 'Sentence: 8392', 'Sentence: 28070', 'Sentence: 40293', 'Sentence: 44552']


In [59]:
pos = list(train_data.Tag)
print(pos[:10])

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O']


In [60]:
for i in range(len(sentences)):
    if (i>0) and (sentences[i]==sentences[i-1]):
        prevtagid = tag2id[pos[i - 1]]
        nexttagid = tag2id[pos[i]]
        count_tags_to_next_tags[prevtagid][nexttagid] += 1

In [61]:
print(count_tags_to_next_tags[0])

[    0 19648  4349     0     0     0   102     0    36     0     1     0
   507     0    74     0     0]


In [62]:
mystartprob = np.zeros((len(tags),))
mytransmat = np.zeros((len(tags), len(tags)))
myemissionprob = np.zeros((len(tags), len(words)))
num_sentences = sum(count_init_tags.values())
sum_tags_to_next_tags = np.sum(count_tags_to_next_tags, axis=1)
for tag, tagid in tag2id.items():
    floatCountTag = float(count_tags.get(tag, 0))
    mystartprob[tagid] = count_init_tags.get(tag, 0) / num_sentences
    for word, wordid in word2id.items():
        myemissionprob[tagid][wordid]= count_tags_to_words.get(tag, {}).get(word, 0) / floatCountTag
    for tag2, tagid2 in tag2id.items():
        mytransmat[tagid][tagid2]= count_tags_to_next_tags[tagid][tagid2] / sum_tags_to_next_tags[tagid]

In [63]:
print(mystartprob.shape)
print(mytransmat.shape)
print(myemissionprob.shape)

(17,)
(17, 17)
(17, 29219)


In [64]:
model = hmm.MultinomialHMM(n_components=len(tags), algorithm='viterbi', random_state=42)
model.startprob_ = mystartprob
model.transmat_ = mytransmat
model.emissionprob_ = myemissionprob

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [65]:
test_data.loc[~test_data['Word'].isin(words), 'Word'] = 'UNKNOWN'   

C:\Temp\ipykernel_12204\3195844866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[~test_data['Word'].isin(words), 'Word'] = 'UNKNOWN'


In [66]:
word_test = list(test_data.Word)
samples = []
for i, val in enumerate(word_test):
    samples.append([word2id[val]])

In [67]:
lengths = []
count = 0
sentences = list(test_data.Sentence)
for i in range(len(sentences)) :
    if (i > 0) and (sentences[i] == sentences[i - 1]):
        count += 1
    elif i > 0:
        lengths.append(count)
        count = 1
    else:
        count = 1

In [68]:
len(lengths)

14387

In [ ]:
ner_predict = model.predict(samples, lengths)